### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing as pp
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, IsolationForest
from sklearn.metrics import  f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
import seaborn as sns
import plotly.express as px
import sweetviz as sv
import pickle
from boruta import BorutaPy
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVR

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

### Helper Functions

In [2]:
alvo='limite_adicional'

def simple_model_test(model, nome):
    #model definition

    # model fit
    model.fit(X_train, y_train)

    #model predict
    y_hat = model.predict(X_test)
    
    # evaluate
    f1 = np.round(f1_score( y_test_, y_hat_ ), 2)
    
    print('{}\n f1: {}'.format(model_name, f1))
    return f1

def send_model(model):
     # model fit
    model.fit(X_train[cols_selected], y_train)
    pickle.dump(model, open('../parameters/model.pkl', 'wb'))
    print('Model submited')
    return None

def target_encoding(df, column):
    target = df.groupby(column)[alvo].mean()
    df[column] = df[column].map(target)
    pickle.dump(target, open(f'../parameters/{column}_encode.pkl', 'wb'))
    print(f'Target Encode to {column}')
    
def frequency_encoding(df, column):
    frequency = df.groupby(column)[alvo].count() / len(df)
    df[column] = df[column].map(frequency)
    pickle.dump(target, open(f'../parameters/{column}_encode.pkl', 'wb'))
    print(f'Frequency Encode to {column}')
    

In [3]:
def encodes_obrigatorios(df, teste=True):
    num_cols = df.select_dtypes('number').drop('id_cliente', axis=1).columns


    map_bool = {'Sim': 1, 'Não': 0}
    map_alvo = {'Conceder': 1, 'Negar': 0}
    string_cols = ['investe_exterior', 'pessoa_polit_exp']

    df['investe_exterior'] = df['investe_exterior'].map(map_bool)
    df['pessoa_polit_exp'] = df['pessoa_polit_exp'].map(map_bool)
    
    if teste==False:
        df[alvo] = df[alvo].map(map_alvo)
    return df

### Load Data 

In [4]:
path = '../data/train.csv'

df_raw = pd.read_csv(path)
df_raw=encodes_obrigatorios(df_raw, teste=False)

df_test = pd.read_csv('../data/test.csv')
df_test=encodes_obrigatorios(df_test, teste=True)

### Data Description

In [5]:
df1 = df_raw.copy()

#### Nulls

In [6]:
df1.isna().sum()

id_cliente                 0
idade                      0
saldo_atual                0
divida_atual               0
renda_anual                0
valor_em_investimentos     0
taxa_utilizacao_credito    0
num_emprestimos            0
num_contas_bancarias       0
num_cartoes_credito        0
dias_atraso_dt_venc        0
num_pgtos_atrasados        0
num_consultas_credito      0
taxa_juros                 0
investe_exterior           0
pessoa_polit_exp           0
limite_adicional           0
dtype: int64

#### Dtypes

In [7]:
df1.dtypes

id_cliente                   int64
idade                        int64
saldo_atual                float64
divida_atual               float64
renda_anual                float64
valor_em_investimentos     float64
taxa_utilizacao_credito    float64
num_emprestimos              int64
num_contas_bancarias         int64
num_cartoes_credito          int64
dias_atraso_dt_venc          int64
num_pgtos_atrasados          int64
num_consultas_credito        int64
taxa_juros                   int64
investe_exterior             int64
pessoa_polit_exp             int64
limite_adicional             int64
dtype: object

### Outliers inspection

In [8]:
df1['pessoa_polit_exp'].value_counts()

0    8917
1     583
Name: pessoa_polit_exp, dtype: int64

In [9]:
df1['limite_adicional'].value_counts()

0    7995
1    1505
Name: limite_adicional, dtype: int64

In [10]:
dashboard=sv.analyze(df1, target_feat='limite_adicional')

                                             |          | [  0%]   00:00 -> (? left)

In [11]:
dashboard.show_html()

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


### Data Filtering

In [12]:
df2 = df1.copy()
#idade - baseado empiricamente
df2.loc[df2['idade']>100, 'idade'] = df2['idade'].median()

# renda anual - baseado no percentil
limit = np.percentile(df2['renda_anual'], 0.95)
df2.loc[df2['idade']>100, 'idade'] 

df2.loc[df2['taxa_juros']>100, 'taxa_juros'] = df2['taxa_juros'].median()

### Feature Engineering

In [13]:
df3 = df2.copy()

df3['negativado'] = np.where(df3['divida_atual']<df3['saldo_atual'], 0, 1)
#df3['renda_mensal'] = np.where(df3['divida_atual']<df3['saldo_atual'], 0, 1)
df3['divida/saldos_atuais']=df3['divida_atual']/(df3['saldo_atual']+df3['valor_em_investimentos'])
df3['divida_atual/renda_mensal']=df3['divida_atual']/(df3['renda_anual']/12)
df3['saldo_liquido']=df3['saldo_atual']+df3['valor_em_investimentos']-df3['divida_atual']
df3['score_credito']=df3['saldo_liquido']*df3['taxa_utilizacao_credito']


df3=df3.drop(['divida_atual','saldo_atual',"valor_em_investimentos","saldo_liquido"], axis=1)

In [14]:
df3.head().T

,0,1,2,3,4
id_cliente,1767.000000,11920.000000,8910.000000,4964.000000,10100.000000
idade,21.000000,40.000000,36.000000,58.000000,35.000000
renda_anual,24196.896360,19227.377960,42822.282230,51786.826000,44626.853460
taxa_utilizacao_credito,31.038763,36.917093,34.561714,31.493561,28.028887
num_emprestimos,6.000000,5.000000,0.000000,0.000000,2.000000
num_contas_bancarias,5.000000,8.000000,3.000000,3.000000,8.000000
num_cartoes_credito,7.000000,5.000000,6.000000,7.000000,7.000000
dias_atraso_dt_venc,21.000000,40.000000,26.000000,12.000000,24.000000
num_pgtos_atrasados,14.000000,23.000000,13.000000,7.000000,10.000000
num_consultas_credito,9.000000,10.000000,3.000000,2.000000,8.000000


### Data Preparation

In [15]:
df4 = df3.copy()

In [16]:
variables_robust=['idade','taxa_juros','num_consultas_credito',"num_pgtos_atrasados","num_cartoes_credito","num_contas_bancarias"]
num_cols = df4.select_dtypes('number').drop(['id_cliente','limite_adicional'], axis=1).columns

#rs_idade=pp.RobustScaler(quantile_range=(0, 95.0))
mms = pp.MinMaxScaler()

df4[num_cols] = mms.fit_transform(df4[num_cols])


In [17]:
def data_preparation(df, teste=True):
    #df[variables_robust]=rs_idade.transform(df[variables_robust])
    df[num_cols]=mms.transform(df[num_cols])
    return df

### COLUMN SELECTION

In [18]:
df5=df4.copy()

In [19]:


y = df5['limite_adicional']
X = df5.drop(['limite_adicional', 'id_cliente'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)

# #training and test dataset for Boruta
x_train_n = X_train.values
y_train_n = y_train.values.ravel()

#define model
rf = RandomForestClassifier(n_jobs=-1, n_estimators=1000)

#define boruta
boruta= BorutaPy(rf, n_estimators='auto', verbose=2, random_state=41).fit(x_train_n, y_train_n)

## Selected cols from boruta
cols_selected = boruta.support_.tolist()

 #best features
X_train_fs = X_train
cols_selected_boruta = X_train_fs.iloc[:, cols_selected].columns.to_list()

#not selected boruta
cols_not_selected_boruta = list(np.setdiff1d(X_train_fs.columns , cols_selected_boruta))

cols_selected_boruta

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	16
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	16
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	16
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	16
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	16
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	16
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	16
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	6
Tentative: 	4
Rejected: 	6
Iteration: 	9 / 100
Confirmed: 	6
Tentative: 	4
Rejected: 	6
Iteration: 	10 / 100
Confirmed: 	6
Tentative: 	4
Rejected: 	6
Iteration: 	11 / 100
Confirmed: 	6
Tentative: 	4
Rejected: 	6
Iteration: 	12 / 100
Confirmed: 	6
Tentative: 	4
Rejected: 	6
Iteration: 	13 / 100
Confirmed: 	6
Tentative: 	4
Rejected: 	6
Iteration: 	14 / 100
Confirmed: 	6
Tentative: 	4
Rejected: 	6
Iteration: 	15 / 100
Confirmed: 	6
Tentative: 	4
Rejected: 	6
Iteration: 	16 / 100
Confirmed: 	6
Tentative: 	4
Rejected: 	6
Iteration:

['renda_anual',
 'num_contas_bancarias',
 'num_cartoes_credito',
 'dias_atraso_dt_venc',
 'num_pgtos_atrasados',
 'num_consultas_credito',
 'taxa_juros',
 'divida/saldos_atuais']

### Model Train

In [20]:
X = df5[cols_selected_boruta]
y = df5[alvo]

In [21]:
import hackday

In [22]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123, stratify=y)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)


100%|██████████| 29/29 [00:08<00:00,  3.61it/s]


In [23]:
models=models.sort_values('F1 Score', ascending=False)
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.88,0.71,0.71,0.87,0.65
LGBMClassifier,0.88,0.73,0.73,0.87,0.26
AdaBoostClassifier,0.88,0.72,0.72,0.87,0.46
ExtraTreesClassifier,0.88,0.70,0.70,0.87,0.57
XGBClassifier,0.87,0.72,0.72,0.86,0.53
BaggingClassifier,0.87,0.70,0.70,0.86,0.17
SGDClassifier,0.84,0.74,0.74,0.85,0.05
DecisionTreeClassifier,0.84,0.70,0.70,0.84,0.05
ExtraTreeClassifier,0.83,0.69,0.69,0.83,0.03


In [24]:
best_model=models.head(1).index[0]
best_model

'RandomForestClassifier'

In [25]:
pipe = clf.models[best_model]
pred = pipe.predict(X_test)

f1_score(y_test, pred, average='micro')

0.8797894736842106

In [26]:
hackday.cv(model_name='LGBM',model=pipe, x_train=X_train, y_train=y_train)

1/5
2/5
3/5
4/5
5/5
LGBM
F1 : 0.87 +/- 0.01
 Precision : 0.64 +/- 0.05
Recall : 0.45 +/- 0.03



In [29]:
types_f1 = ['micro', 'macro', 'samples', 'weighted', 'binary']
for t in types_f1:
    try:
        print(t,f1_score(y_test, pred, average=t))
    except ValueError:
        print(f'Not possible {t}')

micro 0.8797894736842106
macro 0.7423906583809967
Not possible samples
weighted 0.8708777603826184
binary 0.5542544886807182


### Teste

In [30]:
X_submission=df_test.copy()
X_submission = df_test.drop('id_cliente', axis =1)

#renda anual - baseado no percentil
X_submission.loc[X_submission['idade']>100, 'idade'] = X_submission['idade'].median()
limit = np.percentile(X_submission['renda_anual'], 0.95)
X_submission.loc[X_submission['idade']>100, 'idade'] 
X_submission.loc[X_submission['taxa_juros']>100, 'taxa_juros'] = X_submission['taxa_juros'].median()

#Feature engineering
X_submission['negativado'] = np.where(X_submission['divida_atual']<X_submission['saldo_atual'], 0, 1)
X_submission['divida/saldos_atuais']=X_submission['divida_atual']/(X_submission['saldo_atual']+X_submission['valor_em_investimentos'])
X_submission['divida_atual/renda_mensal']=X_submission['divida_atual']/(X_submission['renda_anual']/12)
X_submission['saldo_liquido']=X_submission['saldo_atual']+X_submission['valor_em_investimentos']-X_submission['divida_atual']
X_submission['score_credito']=X_submission['saldo_liquido']*X_submission['taxa_utilizacao_credito']
X_submission=X_submission.drop(['divida_atual','saldo_atual',"valor_em_investimentos","saldo_liquido"], axis=1)

X_submission= data_preparation(X_submission) 
X_submission[num_cols] = mms.fit_transform(X_submission[num_cols])
X_submission=X_submission[cols_selected_boruta]

ids = df_test['id_cliente']

In [31]:
pred = pipe.predict(X_submission)

In [32]:
submission = pd.DataFrame()

In [33]:
submission['id_cliente'] = ids
submission['limite_adicional'] = pred

In [34]:
map_resposta = {0: 'Negar', 1: 'Conceder'}

In [35]:
submission['limite_adicional'] = submission['limite_adicional'].map(map_resposta)

In [36]:
submission.to_csv('../data/submissao.csv', index=False)

In [37]:
pd.read_csv('../data/submissao.csv')

,id_cliente,limite_adicional
0,2,Conceder
1,5,Negar
2,6,Negar
3,8,Negar
4,10,Negar
...,...,...
2995,12484,Negar
2996,12487,Negar
2997,12489,Negar
2998,12495,Conceder
